In [3]:
from qiskit import QuantumCircuit, Aer, execute
import numpy as np
import matplotlib

class QuantumGameInteractive:
    def __init__(self):
        self.board = np.full((4, 4), -1)  # Initialize the board
        self.board[3] = [0, 1, 0, 1]  # Initial configuration
        self.qc = QuantumCircuit(4, 4)  # Quantum circuit for the game
        self.secret_x_pending = [0] * 4  # Track pending secret X gates
        
        self.state_before_hgate = [-1] * 4

        # Apply initial X gates based on the board's configuration
        for qubit in range(4):
            if self.board[3][qubit] == 1:
                self.qc.x(qubit)

    def apply_pending_secret_x(self, qubit):
        while self.secret_x_pending[qubit] > 0:
            self.qc.x(qubit)
            self.secret_x_pending[qubit] -= 1

    def measure_qubit(self, qubit):
        self.apply_pending_secret_x(qubit)  # Apply pending X before measurement

        temp_qc = self.qc.copy()
        temp_qc.measure(qubit, qubit)
        backend = Aer.get_backend('qasm_simulator')
        job = execute(temp_qc, backend, shots=1)
        result = job.result()
        counts = result.get_counts(temp_qc)
        measured_value = int(list(counts.keys())[0][3 - qubit])
        return measured_value

    def update_board(self, player, qubit, action):
        if action == 'X':
            # Mark a secret X gate for future application
            self.secret_x_pending[qubit] += 1
            return  
        
        if action == 'H':
            self.state_before_hgate[qubit] = self.measure_qubit(qubit) 
            self.qc.h(qubit)
            return 

        # Desired outcome aligns with the player's identity (0 or 1)
        desired_outcome = player
        if self.secret_x_pending[qubit]== 0:
            # Measured when no  X gate applied on the qubit. 
            current_state = self.measure_qubit(qubit) 
        elif self.state_before_hgate[qubit]==0 or self.state_before_hgate[qubit]==1:
            current_state= self.state_before_hgate[qubit]
            self.state_before_hgate[qubit] = -1
        else:
            # We only want to measure if there is no X gate applied to that qubit. But if there is 1, we reverse the qubit being measured because the other player when playing their turn won't havre an idea of the X gate. 
            current_state = self.measure_qubit(qubit)
            print(self.secret_x_pending[qubit])
            current_state = 1- current_state
            
        if current_state != desired_outcome:
            self.qc.x(qubit)  # Apply an X gate to achieve desired outcome

        measured_value = self.measure_qubit(qubit)  # Measure again after any adjustments
        # Update the board in the lowest available row for the qubit
        for row in reversed(range(4)):
            if self.board[row][qubit] == -1:
                self.board[row][qubit] = measured_value
                break
        print(f"Player {player} updated the board:")

        print(self.board)

    def play_game(self):
        current_player = 0
        while True:
            print(f"\nPlayer {current_player}'s turn.")
            qubit = int(input("Choose a qubit to interact with (0-3): "))
            action = input("Type 'X' for a secret X gate, 'H' for a secret H gate or 'P' to measure and place: ").upper()

            self.update_board(current_player, qubit, action)

            # We will Implement win condition check and full board check here

            # Switch to the next player
            current_player = 1 - current_player

In [4]:
game = QuantumGameInteractive()
game.play_game()


Player 0's turn.
Player 0 updated the board:
[[-1 -1 -1 -1]
 [-1 -1 -1 -1]
 [ 0 -1 -1 -1]
 [ 0  1  0  1]]

Player 1's turn.
Player 1 updated the board:
[[-1 -1 -1 -1]
 [-1 -1 -1 -1]
 [ 0 -1 -1  1]
 [ 0  1  0  1]]

Player 0's turn.

Player 1's turn.
Player 1 updated the board:
[[-1 -1 -1 -1]
 [ 1 -1 -1 -1]
 [ 0 -1 -1  1]
 [ 0  1  0  1]]

Player 0's turn.


ValueError: invalid literal for int() with base 10: ''